In [1]:
import numpy as np 
from math import log, e 
from scipy.stats import norm 
from calcbsimpvol import calcbsimpvol

# (1) 주식이 q의 연속배당수익률을 지급하는 옵션의 그릭문자 계산 클래스 (표 18.6)
# (2) q를 지수의 배당수익률로 사용하면 유로피언 지수옵션의 그릭문자가 계산된다. (16.4)
# (3) q를 외국의 무위험이자율로 사용하면 유로피언 통화옵션의 그릭문자가 계산된다. 
# 단 통화옵션의 경우 로가 다르다. (밑에 코드에 계산식 적음) (16.5절)
# (4) q=r의 경우 유러피언 선물옵션의 그릭문자(델타, 감마, 세타, 베가)가 계산된다. 
# 단 유로피언 콜선물옵션의 로는(-cT), 풋선물옵션의 로는(-pT)이다.
# 선물옵션의 경우 1976년 블랙이 발표한 블랙-숄즈-머튼 모형을 확장한 선물옵션의 가치를 평가할 수 있는 블랙 모형을 이용한다. 
# S는 F로 대체, q=r로 대체되고 sigma는 선물가격의 변동성이 된다. 
# (보유비용(cost of carry)과 보유편익률(convenience yield)이 시간만의 함수인 경우 선물가격의 변동성은 기초자산의 변동성과 동일하게 된다.)
class BSM: 
    def __init__(self, args): 
        self.asset = str(args[0]) # Option based asset : (1)stock option, (2)stock index option, (3)currency option, (4)future option 
        self.mkt_prc = float(args[1]) # Option Market Price
        self.Type = int(args[2]) # Option Type

        # 선물 옵션의 경우(future option) S=F가 됨. (F는 선물가격) 
        self.S = float(args[3]) # Underlying Price 
        self.K = float(args[4]) # Strike price 
        self.r = float(args[5]) # Interest Rate

        # q는 통화옵션의 경우 외국의 무위험 이자율 rf로 대체된다.
        # q는 주가지수옵션의 경우 지수의 배당수익률이 된다. 
        self.q = float(args[6]) # Dividend Rate 
        self.T = float(args[7]) / 365.0 # Time to Maturity 
        
        # 변동성을 기초자산의 시장가격 self.mkt_prc 을 이용한 내재변동성을 게산해서 구함
        self.sigma = self.imp_vol() # Implied volatility 
        self.sigmaT = self.sigma * np.sqrt(self.T) 
        # 주식의 배당수익률이 q인 경우와 q가 주가지수옵션의 배당수익률인 경우 d1과 d2는 무배당 주식의 옵션과 조금 다르다.
        self.d1 = (np.log(self.S / self.K) + (self.r - self.q + 0.5 * (self.sigma ** 2)) * self.T) / self.sigmaT  
        self.d2 = self.d1 - self.sigmaT 
        
    def imp_vol(self): 
        cp = np.asarray(self.Type) 
        P = np.asarray([self.mkt_prc]) 
        S = np.asarray(self.S) 
        K = np.asarray([self.K]) 
        tau = np.asarray([self.T]) 
        r = np.asarray(self.r) 
        q = np.asarray(self.q) 
        imvol = calcbsimpvol(dict(cp=cp, P=P, S=S, K=K, tau=tau, r=r, q=q)) 
        imvol = np.ndarray.item(imvol) 
            
        return imvol 

    def delta(self): 
        dfq = np.exp(-self.q * self.T) 
            
        if self.Type == 1: 
            return dfq * norm.cdf(self.d1) 
        else: 
            return dfq * (norm.cdf(self.d1) - 1) 

    def gamma(self): 
        return np.exp(-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT) 
        
    def vega(self): 
        return 0.01 * self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * np.sqrt(self.T) 

    # 달력일 기준으로 계산함. 거래일 기준일 경우 365 -> 252    
    def theta(self): 
        df = np.exp(-(self.r * self.T)) 
        dfq = np.exp(-self.q * self.T) 
        tmptheta = (1.0 / 365.0) * (-0.5 * self.S * dfq * norm.pdf(self.d1) * self.sigma / np.sqrt(self.T) + self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) - self.r * self.K * df * norm.cdf(self.Type * self.d2))) 
        return tmptheta 
        
    def rho(self):

        if self.asset == 'stock option' or 'stock index option': 
            
            df = np.exp(-(self.r * self.T))     
            return 0.01 * self.Type * self.K * self.T * df * norm.cdf(self.Type * self.d2)
        
        elif self.asset == 'currecny option':

            rf = self.q         # 통화옵션의 경우 q를 외국의 무위험이자율 rf로 둠
            df_foreign = np.exp(-(rf * self.T))
            df_domestic =  np.exp(-(self.r * self.T))

            # 국내이자율 r에 대한 rho = 0.01 * self.Type * self.K * self.T * df * norm.cdf(self.Type * self.d2)
            # 외국이자율 rf에 대한 rho = (-1) * 0.01 * self.Type * self.T * df * self.S * norm.cdf(self.Type * self.d1)

            df = df_domestic
            rho_domestic = 0.01 * self.Type * self.K * self.T * df * norm.cdf(self.Type * self.d2)

            df = df_foreign
            rho_foreign = (-1) * 0.01 * self.Type * self.T * df * self.S * norm.cdf(self.Type * self.d1)
            
            return (rho_domestic, rho_foreign)

        elif self.aseet == 'future option':
            
            # self.q == self. r이 됨. 따라서 d1, d2, c, p값 모두 변화. self.S == F 즉 선물가격이 됨
            F = self.S
            df = np.exp(-(self.r * self.T)) 
            # c = df * (F * norm.cdf(d1) - K * norm.cdf(self.Type*d2))
            # p = df * (K * norm.cdf(self.Type*d2) - F * norm.cdf(self.Type*d1))
            # rho = -cT or -pT


            return (-1) * 0.01 * self.Type * df * (F * norm.cdf(d1) - K * norm.cdf(self.Type*d2)) * self.T
        else : 
            print("옵션의 기초자산이 무엇인지 정확히 입력하세요")


In [2]:
# (2) q가 지수의 배당수익률인 주가지수옵션의 그릭문자 계산
# 배당수익률의 추정치가 필요하면 코스피의 배당수익률의 경우 다음 사이트에서 값을 찾을 수 있다.
# 과거 배당수익률 계산된 것 http://kosis.kr/statHtml/statHtml.do?orgId=343&tblId=DT_343_2010_S0032
# 배당수익률 추이 보기 http://marketdata.krx.co.kr/mdi#document=13010105
# 만약 배당수익률의 추정치를 계산하고 싶다면(ex. 아메리칸옵션의 가치를 평가하므로) 행사가격과 만기가 동일한 콜옵션과 풋옵션이 이용될 수 있다.
# 배당수익률 q를 지급하는 주식에 대한 옵션의 풋-콜 패리티 c + K*e**(-r*T) = p + S*e**(-q*T) 에서 q = -1/T*ln((c-p+K*e**(-r*T))/S)를 이용
# 특정 행사가격과 만기의 옵션을 이용하여 계산된 q의 추정치는 신뢰하기 어렵지만 많은 쌍의 콜옵션과 풋옵션으로부터 결과를 결합하여 추정하면 시장이 가정하는 배당수익률이 비교적 정확하게 계산됨

# 옵션 행사가,현재가 볼 수 있는 곳 http://marketdata.krx.co.kr/mdi#document=060201

# 코스피 200의 경우 대략 parameter을 다음과 같이 잡을 수 있음. 2020/08/07 PM 06:57:22 (20분 지연 정보)기준)
# 2020/08/07 PM 06:57:22 (20분 지연 정보)기준)
# 코스피200 C 202008 320.0 8월 둘째주 목요일 만기 행사가 320이 거래량이 제일 많았음. 거래량이 많은거 기준으로 하자.
# 하지만 코스피200 C 202009 330.0 즉 9월 만기로 계산해보고자 함. (8월 만기는 너무 짧아서 9월 중 거래량 제일 많은 것 선택)
option_asset = 'stock index option'
mkt_prc = 1.65 # Market Price 코스피200 C 202009 330.0 즉 9월 만기의 현재가. 계산해보고자 함. (8월 만기는 너무 짧아서)
option_type = 1 # Option Type 
S = 312.57 # Underlying Price (kospi200 : 2020.08.07 종가)
K = 330.00 # Strike Price 
q = 0.0195 # Dividend Rate (kospi200 배당수익률 추이 1.95% 2020.08.06)
r = 0.0075 # Interest Rate (risk free rate : CD금리(91일) 2020.08.27 종가)
T = 1/12 # Time to Maturity (9월 둘째주 만기인데 대략 1달로 계산)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_kospi_200 = BSM(args) 
imp_vol = BSM_kospi_200.imp_vol() 
delta = BSM_kospi_200.delta() 
gamma = BSM_kospi_200.gamma() 
vega = BSM_kospi_200.vega() 
theta = BSM_kospi_200.theta() 
rho = BSM_kospi_200.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  3.75669225
Delta =  0.1767969
Gamma =  0.01462293
Vega =  0.01225351
Theta =  -27.61774856
Rho =  0.0001224


In [3]:
# (2) q가 지수의 배당수익률인 주가지수옵션의 그릭문자 계산
# S&P 500에 대한 유러피언 콜옵션 (만기는 2개월) 16장 예제 16.1
option_asset = 'stock index option'
mkt_prc = 52 # Market Price (일단 여기서는 가상으로 옵션가격을 잡음. 만약 지수의 변동성이 20%와 나머지 parameter로 유로피언 콜옵션을 블랙-숄즈-머튼 모형을 이용해서 가치평가할 경우 콜옵션의 가격은 51.83달러가 됨) 
option_type = 1 # Option Type 
S = 930 # Underlying Price 
K = 900 # Strike Price 
q = 0.03 # Dividend Rate (1개월 동안의 배당수익률이 0.2%, 다음 1개월 동안의 배당수익률이 0.3% 이면 총 배당수익률은 2개월 동안 0.5% 이고 이는 연 3%에 해당함)
r = 0.08 # Interest Rate (risk free rate)
T = 2/12 # Time to Maturity (2개월 만기)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_sp_500 = BSM(args) 
imp_vol = BSM_sp_500.imp_vol() 
delta = BSM_sp_500.delta() 
gamma = BSM_sp_500.gamma() 
vega = BSM_sp_500.vega() 
theta = BSM_sp_500.theta() 
rho = BSM_sp_500.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  4.48391122
Delta =  0.65185772
Gamma =  0.00414855
Vega =  0.07346408
Theta =  -98.89357739
Rho =  0.00253072


In [4]:
# (3) q를 외국의 무위험이자율로 사용하는 유로피언 통화옵션의 그릭문자 계산
# 영국 파운드화에 대한 4개월 만기 유러피언콜옵션 고려. 16장 예제 16.2
# 즉, domestic country = America, foreign country : England
option_asset = 'currency option'
mkt_prc = 0.043 # Market Price : 4.3 cent
option_type = 1 # Option Type 
S = 1.6 # Underlying Price : 현재 환율
K = 1.6 # Strike Price : 행사가격 환율
q = 0.11 # Dividend Rate but risk free rate of foreign country in currecny option (영국의 무위험 이자율)
r = 0.08 # Interest Rate (risk free rate of domestic country) (미국의 무위험 이자율)
T = 4/12 # Time to Maturity (4개월 만기)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_currency = BSM(args) 
imp_vol = BSM_currency.imp_vol() 
delta = BSM_currency.delta() 
gamma = BSM_currency.gamma() 
vega = BSM_currency.vega() 
theta = BSM_currency.theta() 
rho = BSM_currency.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  2.2309336
Delta =  0.51323192
Gamma =  3.69594957
Vega =  0.00019277
Theta =  -0.06443151
Rho =  7.11e-06


In [5]:
# (4) q=r의 경우 유러피언 선물옵션의 그릭문자 계산. 맨 위에 주의점을 적은 주석 참고
# 만기가 6개월인 은선물의 유러피언 선물옵션 고려. 17장 예제 17.5

option_asset = 'future option'
mkt_prc = 0.56 # Market Price : 은선물의 콜옵션 가격 0.56 dollar
option_type = 1 # Option Type 
S = 8.0 # Underlying Price : 은 선물의 현재 가격 8.0 dollar
K = 8.5 # Strike Price : 은 선물 콜옵션의 행사가격 8.50 dollar
q = 0.10 # 유러피언 선물옵션의 경우 q=r임. 여기서는 포맷을 유지하기 위해 q,r둘다 parameter로 넘겨주되, 같은 값으로 설정
r = 0.10 # Interest Rate 6개월 만기 무위험이자율 연 10%
T = 6/12 # Time to Maturity (6개월 만기)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_futures = BSM(args) 
imp_vol = BSM_futures.imp_vol() 
delta = BSM_futures.delta() 
gamma = BSM_futures.gamma() 
vega = BSM_futures.vega() 
theta = BSM_futures.theta() 
rho = BSM_futures.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  6.46039106
Delta =  0.44664479
Gamma =  0.20666399
Vega =  0.00117053
Theta =  -0.75605155
Rho =  4.128e-05


In [6]:
# (4) q=r의 경우 유러피언 선물옵션의 그릭문자 계산. 맨 위에 주의점을 적은 주석 참고
# 만기가 4개월인 원유의 유러피언 풋선물옵션 고려. 17장 예제 17.6

option_asset = 'future option'
mkt_prc = 1.00 # Market Price : 원유선물의 풋옵션 가격 가상으로 잡아줌. 1 dollar. 시장가격으로 내재변동성을 구하기 위함. 
               # 선물가격의 변동성이 25% 라 가정하고 나머지 밑 paramter으로 블랙 모형을 통해 이론가를 계산했을 때 1.12 dollar 가 나옴
option_type = -1 # Option Type 
S = 20 # Underlying Price : 원유 선물의 현재 가격 20 dollar
K = 20 # Strike Price : 은 선물 콜옵션의 행사가격 8.50 dollar
q = 0.09 # 유러피언 선물옵션의 경우 q=r임. 여기서는 포맷을 유지하기 위해 q,r둘다 parameter로 넘겨주되, 같은 값으로 설정
r = 0.09 # Interest Rate 4개월 만기 무위험이자율 연 9%
T = 4/12 # Time to Maturity (4개월 만기)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_futures = BSM(args) 
imp_vol = BSM_futures.imp_vol() 
delta = BSM_futures.delta() 
gamma = BSM_futures.gamma() 
vega = BSM_futures.vega() 
theta = BSM_futures.theta() 
rho = BSM_futures.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  4.15037448
Delta =  -0.47495891
Gamma =  0.15871214
Vega =  0.00240626
Theta =  -1.49778806
Rho =  -9.588e-05


In [7]:
# (4) q=r의 경우 유러피언 선물옵션의 그릭문자 계산. 맨 위에 주의점을 적은 주석 참고
# 금의 현물가격에 대한 만기가 6개월인 유러피언 콜선물옵션 고려. 17장 예제 17.7
# 이 옵션은 6개월 만기 선물계약에 대한 6개월 만기 유러피언 콜선물옵션과 동일함

option_asset = 'future option'
mkt_prc = 88   # Market Price : 금 선물의 콜옵션 가격을 가상으로 88 dollar 로 설정. 시장가격으로 내재변동성을 구하기 위함. 
               # 선물가격의 변동성이 20% 라 가정하고 나머지 밑 paramter으로 블랙 모형을 통해 이론가를 계산했을 때 88.37 dollar 가 나옴
option_type = 1 # Option Type 
S = 1240 # Underlying Price : 6개월 만기 금선물 가격 1240 dollar
K = 1200 # Strike Price : 금 선물 콜옵션의 행사가격 1200 dollar
q = 0.05 # 유러피언 선물옵션의 경우 q=r임. 여기서는 포맷을 유지하기 위해 q,r둘다 parameter로 넘겨주되, 같은 값으로 설정
r = 0.05 # Interest Rate 6개월 만기 무위험이자율 연 5%
T = 6/12 # Time to Maturity (6개월 만기)

args = [option_asset, mkt_prc, option_type, S, K, r, q, T] 

BSM_futures = BSM(args) 
imp_vol = BSM_futures.imp_vol() 
delta = BSM_futures.delta() 
gamma = BSM_futures.gamma() 
vega = BSM_futures.vega() 
theta = BSM_futures.theta() 
rho = BSM_futures.rho() 

print('Implied Vol = ', round(imp_vol, 8)) 
print('Delta = ', round(delta, 8)) 
print('Gamma = ', round(gamma, 8)) 
print('Vega = ', round(vega, 8)) 
print('Theta = ', round(theta, 8)) 
print('Rho = ', round(rho, 8))

Implied Vol =  3.67191192
Delta =  0.62138206
Gamma =  0.00225665
Vega =  0.17453278
Theta =  -64.07484361
Rho =  0.0093495
